# ENV

In [1]:
from pprint import pprint
from llama_index.core.utils import print_text
from dotenv import load_dotenv
from src.core.utils.settings import load_settings

load_dotenv("local.env")
settings = load_settings()

E:\Code\Miniconda\conda\envs\torch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Models

In [8]:
from src.core.modules.models import GoogleLLM, GoogleEmbedding
from llama_index.core import Settings

emb_model = GoogleEmbedding(api_key=settings.google_ai.api_key)
llm = GoogleLLM(api_key=settings.google_ai.api_key)
Settings.embed_model = emb_model
Settings.llm = llm

# Query

In [ ]:
query = "Who is Japanese singer Yuuri?"

In [ ]:
response = llm.predict(query)
pprint(response)

# RAG

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [ ]:
# Prepare documents
documents = SimpleDirectoryReader(input_dir="data").load_data()

In [ ]:
# Create index
index = VectorStoreIndex.from_documents(documents)
# Save index to disk
index.storage_context.persist("index")

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

In [ ]:
# Load index
storage_context = StorageContext.from_defaults(persist_dir="index")
index = load_index_from_storage(storage_context)

In [ ]:
from src.core.modules.response_synthesizers import google_response_synthesizer

In [ ]:
# Query engine
query_engine = index.as_query_engine(
    response_synthesizer=google_response_synthesizer(llm=llm)
)

In [ ]:
query = "Who is Japanese singer Yuuri?"

In [ ]:
response = query_engine.query(query)
pprint(response.response)

# Agent

In [9]:
import wikipedia
from llama_index.core.tools import FunctionTool

In [10]:
# Define tools
def search_info_from_wikipedia(prompt: str) -> list[str]:
    """
    Do a wikipedia search for provided `prompt`
    Returns a list of ten wikipedia titles that matches the prompt
    """
    return wikipedia.search(prompt)


def get_wikipedia_content_from_title(title: str) -> str:
    """ Functions returns content of wikipedia page based on provided title """
    page = wikipedia.page(title)
    return page.content

In [11]:
search_tool = FunctionTool.from_defaults(fn=search_info_from_wikipedia)
content_tool = FunctionTool.from_defaults(fn=get_wikipedia_content_from_title)

In [12]:
gemini = GoogleLLM(
    api_key=settings.google_ai.api_key,
    is_chat_model=True
)

In [13]:
from src.core.modules.agents import GeminiReAct

In [14]:
agent = GeminiReAct.from_tools([search_tool, content_tool], llm=llm, verbose=True)

In [15]:
response = agent.chat("Who is the Japanese singer Yuuri?")
pprint(response.response)

> Prompt input to the LLM: 
Role: user
Content: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
> Tool Name: search_info_from_wikipedia
Tool Description: search_info_from_wikipedia(prompt: str) -> list[str]

    Do a wikipedia search for provided `prompt`
    Returns a list of ten wikipedia titles that matches the prompt
    
Tool Args: {"type": "object", "properties": {"prompt": {"title": "Prompt", "type": "string"}}, "required": ["prompt"]}

> Tool Name: get_wikipedia_content_from_title
Tool Description: get_wikipedia_content_from_title(title: str) -> str
 Functions returns content of wikipedia page based on 

# Tools

In [16]:
from llama_index.core.tools import FunctionTool

In [17]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

In [18]:
tool = FunctionTool.from_defaults(fn=multiply)

In [19]:
print_text(tool.metadata.description, color='green')

multiply(a: int, b: int) -> int
Multiply two integers and returns the result integer

In [20]:
print_text(tool.metadata.name, color='green')

multiply

In [21]:
print_text(tool.metadata.get_parameters_dict(), color='green')

{'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b']}

# ReAct system prompt

In [22]:
from src.core.modules.agents.prompt_formatter import REACT_CHAT_SYSTEM_HEADER

In [23]:
print_text(REACT_CHAT_SYSTEM_HEADER)

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 

# ReAct Agent Flow

## Setup new Agent

In [24]:
from src.core.modules.agents import GeminiReAct

In [25]:
llm = GoogleLLM(api_key=settings.google_ai.api_key)

In [26]:
agent = GeminiReAct.from_tools([tool], llm=llm)

## Get tasks

In [27]:
task = agent.create_task("What is 2*8?")

In [28]:
pprint(task)

Task(task_id='e4a83afb-834e-4bb9-8971-9984aecd9278', input='What is 2*8?', memory=ChatMemoryBuffer(token_limit=23040, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history'), callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x00000210615F0A00>, extra_state={'sources': [], 'current_reasoning': [], 'new_memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')})


In [29]:
step_queue = agent.state.get_step_queue(task.task_id)
step = step_queue.popleft()

In [30]:
pprint(step)

TaskStep(task_id='e4a83afb-834e-4bb9-8971-9984aecd9278', step_id='8024648b-cd03-45ac-b5e9-b39d6e682a66', input='What is 2*8?', step_state={'is_first': True}, next_steps={}, prev_steps={}, is_ready=True)


## Get tools

In [31]:
tools = agent.agent_worker.get_tools(task.input)

In [32]:
pprint(tools)

## Modify task and prompt according to step input

In [33]:
print_text(step.input)

What is 2*8?

In [34]:
from llama_index.core.agent.react.step import add_user_step_to_reasoning

In [35]:
add_user_step_to_reasoning(
    step=step,
    memory=task.extra_state["new_memory"],
    current_reasoning=task.extra_state["current_reasoning"],
    verbose=True
)

In [36]:
print_text(task.extra_state["new_memory"])

token_limit=3000 tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all') chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, content='What is 2*8?', additional_kwargs={})]}) chat_store_key='chat_history'

In [37]:
input_chat = agent.agent_worker._react_chat_formatter.format(
    tools,
    chat_history=task.memory.get() + task.extra_state["new_memory"].get_all(),
    current_reasoning=task.extra_state["current_reasoning"],
)

In [38]:
for chat in input_chat:
    print_text(chat["role"] + ': ', color='red')
    print_text(chat["parts"][0], color='green')

user: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
> Tool Name: multiply
Tool Description: multiply(a: int, b: int) -> int
Multiply two integers and returns the result integer
Tool Args: {"type": "object", "properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"]}



## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of multiply) if using a tool.
Action Inpu

## Get response from LLM

In [39]:
chat_response = agent.agent_worker._llm.complete(input_chat)

In [40]:
# Format the chat_response
from llama_index.core.base.llms.types import ChatResponse, ChatMessage, MessageRole
chat_response = ChatResponse(
    message=ChatMessage(
        content=chat_response.text
    )
)

In [41]:
print_text("LLM: ", color='red')
print_text(f"{str(chat_response.message.content)}", color='green')

LLM: Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {"a": 2, "b": 8}

## How function calling works

In [42]:
_, current_reasoning, is_done = agent.agent_worker._extract_reasoning_step(chat_response)

In [43]:
pprint(current_reasoning[-1])

ActionReasoningStep(thought='The current language of the user is: english. I need to use a tool to help me answer the question.', action='multiply', action_input={'a': 2, 'b': 8})


In [44]:
from llama_index.core.agent.react.types import ActionReasoningStep
from typing import cast

In [45]:
reasoning_step = cast(ActionReasoningStep, current_reasoning[-1])

In [46]:
pprint(reasoning_step)

ActionReasoningStep(thought='The current language of the user is: english. I need to use a tool to help me answer the question.', action='multiply', action_input={'a': 2, 'b': 8})


In [47]:
tool_dict = {
    tool.metadata.get_name(): tool for tool in tools
}

In [48]:
pprint(tool_dict)

{'multiply': <llama_index.core.tools.function_tool.FunctionTool object at 0x00000210615F3DF0>}


In [49]:
tool_output = tool_dict[reasoning_step.action].call(**reasoning_step.action_input)

In [50]:
pprint(tool_output)

ToolOutput(content='16', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 8}}, raw_output=16, is_error=False)
